In [1]:
#Estimation of reaction rates of bacterial serotonin production from published data, park et al, 2014
#E.coli concentration at OD600=8e8 cells/ml, 
#initial concentration of tryptophan: 3 mM
#initial concentration of serotonin: 0 mM

#Assume 1st order, irreversible reaction (enzymatic, at low concentrations can assume non-MM) 
#Two step reaction, with one rate limiting step, will assume one reaction rate for overall reaction: A -> B -> C

In [5]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

timesRXN = np.array([0.0, 3.0, 6.0, 9.0, 12., 15., 24.]) #hours
serConcRXN = np.array([0.0, 0.055612, 0.124844, 0.147543, 0.173079, 0.181591, 0.209965]) #mM
trpConcRXN = 3.0 - serConcRXN #mM




In [15]:
import scipy.optimize
import scipy.integrate

# Adapted Professor West's solution to regression assignment

def nonelementary(times, k, n):
    """
    fitting to a R_a = k*C_a^n
    """
    M = len(times)
    y_data = np.zeros(M)
    for i in range(M):
        t = times[i]
        
        def dcAdt(cA, t):
            return -1.*k*cA**n

        cAo=3 #mmol/L
        result = scipy.integrate.odeint(dcAdt, cAo, [0,t])
        cA = result[-1,0]
        y_data[i] = cA
    return y_data

    
def Michaelis_Menten(times,K_m,v_max):
    """
    Trying out a Michaelis_Menten fit for reaction kinetics of tryptophan to serotonin
    """
    M = len(times)
    y_data = np.zeros(M)

    for i in range(M):
        t = times[i]
        
        def dcAdt(cA, t):
            return -1.0*v_max*cA/(cA+K_m)
    
        cAo=3 #mmol/L
        result = scipy.integrate.odeint(dcAdt, cAo, [0,t])
        cA = result[-1,0]
        y_data[i] = cA
    return y_data

startingGuessNE = (1, 1)
startingGuessMM = (1, 1)

ne_optimal_parameters, ne_covariance = scipy.optimize.curve_fit(nonelementary, timesRXN, trpConcRXN, 
                                                                p0=startingGuessNE)

mm_optimal_parameters, mm_covariance = scipy.optimize.curve_fit(Michaelis_Menten, timesRXN, trpConcRXN, 
                                                                p0=startingGuessMM)


def report(optimal_parameters, covariance):
    "Make this a function so we can reuse it in cells below"
    parameter_errors = numpy.sqrt(numpy.diag(covariance))
    for i in range(len(optimal_parameters)):
        print("Parameter {}: {} +/- {} (1 st. dev.)".format(i,
                                                            optimal_parameters[i],
                                                            parameter_errors[i]))
print('\n\nNonelementary form of equation \n' + '-'*40 + '\n')
report(ne_optimal_parameters, mm_covariance)
print('\n\nMichaelis_Menten form of equation \n' + '-'*40 + '\n')
report(mm_optimal_parameters, mm_covariance)


/Users/Cain/anaconda2/envs/py3/lib/python3.5/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)




Nonelementary form of equation 
----------------------------------------

Parameter 0: 12.850713303133942 +/- 184631.72405402744 (1 st. dev.)
Parameter 1: 384.63057522365943 +/- 724.4091552462596 (1 st. dev.)


Michaelis_Menten form of equation 
----------------------------------------

Parameter 0: 2150.326250841565 +/- 184631.72405402744 (1 st. dev.)
Parameter 1: 8.450878190543836 +/- 724.4091552462596 (1 st. dev.)


/Users/Cain/anaconda2/envs/py3/lib/python3.5/site-packages/scipy/optimize/minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [4]:
#above assumes a concentration of 8e8 bacteria per mL
#Based on average intestinal (small and large) volume of 130 mL
#and total microbes in the body, 10^14, concentrations could be high as 10^14/130 per mL
#but not all produce serotonin

serProdMicrobeMax = 10e14/130 #bacteria per mL
serProdMicrobeMin = serProdMicrobeMax*0.000001


#Adjustments to k value based on bacterial concentration 
#serConcRXNMin = (serProdMicrobeMin/8e8)*serConcRXN
#trpConcRXNMin = 3.0 - serConcRXNMin
#serConcRXNMax = (serProdMicrobeMax/8e8)*serConcRXN
#trpConcRXNMax = 3.0 - serConcRXNMax

#does not account for change in RATE how to account for differences based on differences in number of bacteria from this data?


optimal_parametersMin, covarianceMin = scipy.optimize.curve_fit(RXNfunction, timesRXN, trpConcRXNMin)
optimal_parametersMax, covarianceMax = scipy.optimize.curve_fit(RXNfunction, timesRXN, trpConcRXNMax)

report(optimal_parametersMin, covarianceMin)
report(optimal_parametersMax, covarianceMax)

plt.plot(timesRXN, trpConcRXNMin)
plt.plot(timesRXN, trpConcRXNMax)
plt.plot(x_many_points, RXNfunction(x_many_points, *optimal_parametersMin), label='fitted')
plt.plot(x_many_points, RXNfunction(x_many_points, *optimal_parametersMax), label='fitted')
    
plt.show()
    


NameError: name 'trpConcRXNMin' is not defined